# Imports

In [1]:
from OPqkdsim.channel import *
from OPqkdsim.measure import *
from OPqkdsim.polarization import *
from OPqkdsim.source import *
from OPqkdsim.timeline import *
from OPqkdsim.random import *
from OPqkdsim.classpostpro import *

In [2]:
# Initialize Timeline

timeline = Timeline()
shots=10
# Define bit lists and modulation lists
rng = CRNG(method="crypto")
alice_bit_list = rng.generate_bits(shots)
alice_modulation_list = rng.generate_basis(shots)
bob_modulation_list = rng.generate_basis(shots)
bob_bit_list = [] 

In [3]:
# Initialize Alice Components

laser = DFBLaser(timeline, shots=shots)
pc1 = PolarizationController(timeline)  # Alice's polarization control
pm1 = AlicePolarizationModulator(timeline, modulation_list=alice_modulation_list, bit_list=alice_bit_list)
attenuator = VariableOpticalAttenuator(timeline, attenuation_dB=60)  # Attenuates to single-photon level # why 60db ?

# Quantum channel

channel = QuantumChannel(timeline)  


In [4]:
# Initialize Bob Components

pc2 = PolarizationController(timeline, compensate=True)  # Bob's first polarization control
pm2 = BobPolarizationModulator(timeline, modulation_list=bob_modulation_list)
pc3 = PolarizationController(timeline)  # Final polarization correction

# Create Detectors

detector_HV = SinglePhotonDetector(timeline, name="Detector_H", bit_list=bob_bit_list)
#detector_V = SinglePhotonDetector(timeline, name="Detector_V", bit_list=bob_bit_list)
detector_D = SinglePhotonDetector(timeline, name="Detector_D+", bit_list=bob_bit_list)
#detector_D_minus = SinglePhotonDetector(timeline, name="Detector_D-", bit_list=bob_bit_list)

# Create PBS

pbs = PolarizationBeamSplitter(timeline, detector_HV, detector_D, bob_modulation_list)


In [5]:
# Connect Components Using Timeline’s Publish-Subscribe Model

timeline.subscribe(laser, pc1.process_signal)
timeline.subscribe(pc1, pm1.process_signal)
timeline.subscribe(pm1, attenuator.process_signal)
timeline.subscribe(attenuator, channel.propagate_signal)
timeline.subscribe(channel, pc2.process_signal)
timeline.subscribe(pc2, pm2.process_signal)
timeline.subscribe(pm2, pc3.process_signal)
timeline.subscribe(pc3, pbs.process_signal)
timeline.subscribe(pbs, detector_HV.detect_photon)
timeline.subscribe(pbs, detector_D.detect_photon)

In [6]:
# Start Simulation by Emitting Light from the Laser
laser.initialize_laser()
timeline.execute_events()

[1.74034e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:10
[1.74034e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:9
[1.74034e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74034e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:8
[1.74034e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74034e+09 s] Phase Modulated: Basis=1, Bit=0, φ=1.57080 rad
[1.74034e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:7
[1.74034e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74034e+09 s] Phase Modulated: Basis=1, Bit=0, φ=1.57080 rad
[1.74034e+09 s] VOA applied 60.00000 dB attenuation. New Power: 1.20000e-13 W
[1.74034e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:6
[1.74034e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74034e+09 s] Phase Modulated: Basis=0, Bit=0, φ=0.00000 rad
[1.74034e+09 s] VOA applied 60.00000 dB attenuation.

In [7]:
# matching_bits = []
# for alice_basis, bob_basis, alice_bit, bob_bit in zip(alice_modulation_list, bob_modulation_list, alice_bit_list, bob_bit_list):
#     if alice_basis == bob_basis:  # Matching bases
#         matching_bits.append(alice_bit)

# # Display Results
# print("\n--- BB84 Protocol Results ---")
# print(f"Alice's Bits: {alice_bit_list}")
# print(f"Alice's Bases: {alice_modulation_list}")
# print(f"Bob's Bases: {bob_modulation_list}")
# print(f"Bob's Received Bits: {bob_bit_list}")
# print(f"Final Key (Matching Basis): {matching_bits}")

In [12]:
key_length = 10**1
er = lpdcER(key_length, code_rate=0.8)
corrected_key = er.reconcile( np.array(alice_bit_list), np.array(alice_modulation_list), 
                            np.array(bob_bit_list), np.array(bob_modulation_list))
print("Final corrected_key :", corrected_key)
pa = UhashPA(output_length=256)
secure_key = pa.amplify(corrected_key)

print("Final secure key:", secure_key)

# QBER Calculation
# qber_calculator = QBERCalculator(sifted_key_length=key_length, sample_bits=5000)
# qber = qber_calculator.calculate_qber(alice_bit_list, bob_bit_list)
# stat_error = qber_calculator.calculate_statistical_error()
# approx_qber = qber_calculator.calculate_approximated_qber()

# print(f"QBER: {qber:.6f}, Statistical Error: {stat_error:.6f}, Approximated QBER: {approx_qber:.6f}")


ValueError: matmul: dimension mismatch with signature (n,k=10),(k=6,1?)->(n,1?)